In [1093]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
from geopy import distance as geo
from sklearn import metrics
#Import for project
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score#Import for project
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import math
import datetime as dt 

In [1094]:
#load data set 
df = pd.read_csv(r"C:\Users\manue\git\PDS\Project\PDS_Project\output_data\transform_DF", index_col=0)
weather = pd.read_csv(r"C:\Users\manue\git\PDS\Project\PDS_Project\data\frankfurt_weather_data2019.csv", sep=",", index_col=0)

# calculate durantion between center point of downtown and
down_town = pd.DataFrame(data=[["point A", 8.683319091796875, 50.110285930959314]],
                                columns=["Describtion", "long", "latitude"])
     
    
location_Uni = pd.DataFrame(data=[["point A", 8.692339207868319, 50.130519449999994]],
                                columns=["Describtion", "long", "latitude"])
 

In [1095]:
    # calculate durantion between center point of univer. and trip end classifi
    distance_begin = pd.Series(index=df.index, data=list(map(
        lambda x: geo.distance(tuple(location_Uni.loc[0]["long":"latitude"]),
                               tuple((df.loc[x]["Start_Longitude"], df.loc[x]["Start_Latitude"]))).km, df.index)))
    distance_end = pd.Series(index=df.index, data=list(map(
        lambda x: geo.distance(tuple(location_Uni.loc[0]["long":"latitude"]),
                               tuple((df.loc[x]["End_Longitude"], df.loc[x]["End_Latitude"]))).km, df.index)))
    
    distance_DT = pd.Series(index=df.index, data=list(map(
        lambda x: geo.distance(tuple(down_town.loc[0]["long":"latitude"]),
                               tuple((df.loc[x]["End_Longitude"], df.loc[x]["End_Latitude"]))).km, df.index)))
    
    

In [1136]:
    # creat data frame for target 
    Dist = pd.DataFrame(distance_begin)
    
    # Create target column
    Dist["end_d"] = distance_end
    Dist["uni"] = Dist[0] > Dist["end_d"]
    Dist["uni"] = Dist["uni"].astype(int)
    df = Dist.join(df)

In [1097]:
    df["weather"] = 0
    
    # Format date to other format "2019-20-01 00:00:00 to 2019200100"
    df["date"] = df["Start_Time"].astype(str).str.replace("-","")
    df["date"] = df["date"].str.replace(":*:.*","")
    df["date"] = df["date"].str.replace("\s","")
    
    # set temp to datetype string
    weather.index = weather.index.astype(str)

    #Join weather dates to predictor matrix 
    df = df.join(weather, on="date" ,how="inner")
    
    #create weather feature 
    df["weather"] = df["hourly temperatur"] > 5 & (df["Rainfall"] == 0)
    df.drop(columns=["relative humidity", "Rainfall", "SD_SO", "V_VV"],inplace=True)
    df["weather"] = df["weather"].astype(int)
    df
    
    
    
    

,0,end_d,uni,Bike_number,Start_Time,End_time,Start_Longitude,Start_Latitude,End_Longitude,End_Latitude,Start_Station,End_Station_number,Weekday,Duration,Bikes_on_position,End_Bikes,Zip_codes,weather,date,hourly temperatur
13,15.685492,15.576618,1,11451,2019-05-30 15:42:00,2019-05-30 17:22:00,8.553437,50.101748,8.554450,50.101711,4291.0,0.0,True,100.0,5,1,65929,1,2019053015,20.1
19571,15.685492,15.572969,1,38089,2019-05-30 15:43:00,2019-05-30 17:20:00,8.553437,50.101748,8.554479,50.101733,4291.0,0.0,True,97.0,4,1,65929,1,2019053015,20.1
28319,4.388720,8.134838,0,38137,2019-05-30 15:42:00,2019-05-30 16:08:00,8.659075,50.108778,8.637696,50.081042,0.0,0.0,True,26.0,1,1,60329,1,2019053015,20.1
39943,4.268353,6.818889,0,38198,2019-05-30 15:05:00,2019-05-30 15:16:00,8.668331,50.100153,8.649362,50.086095,4209.0,4135.0,True,11.0,4,5,60596,1,2019053015,20.1
52183,2.053770,6.069914,0,38267,2019-05-30 15:06:00,2019-05-30 15:28:00,8.679049,50.117486,8.643708,50.104957,4237.0,4195.0,True,22.0,5,4,60322,1,2019053015,20.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152507,2.673466,2.551191,1,38993,2019-01-21 10:00:00,2019-01-21 10:09:00,8.695486,50.106433,8.707100,50.112704,4130.0,4217.0,True,9.0,5,5,60594,0,2019012110,-3.2
153342,1.787325,2.878442,0,38995,2019-12-25 12:13:00,2019-12-25 12:23:00,8.687546,50.115009,8.671153,50.115329,4247.0,4243.0,True,10.0,6,1,60313,1,2019122512,8.2
153360,3.078361,1.829000,1,38996,2019-02-14 02:06:00,2019-02-14 02:29:00,8.674521,50.109030,8.708392,50.126528,4242.0,4255.0,True,23.0,2,5,60329,0,2019021402,0.9
153682,0.820865,1.297919,0,38997,2019-04-08 00:27:00,2019-04-08 00:33:00,8.692680,50.123068,8.701279,50.122879,4250.0,4251.0,True,6.0,5,4,60316,1,2019040800,13.6


In [1098]:
    # stardistanz/extract, hour, month,
    df["date"] = df["Start_Time"].astype(str).str.replace("-", "")
    df["date"] = df["date"].str.replace(":*:.*", "")
    df["date"] = df["date"].str.replace("\s", "")

    # Create intervalls for hours
    df["hour"] = df["date"].astype(str).str.extract("([0-9]{2}$)").astype(int)

    # exctract month
    df["month"] = df["date"].astype(str).str.extract("(^[0-9]{6})").astype(int)
    df["month"] = df["month"].astype(str).str.extract("([0-9]{2}$)").astype(int)
    
    # extract day of year and week
    df["day_of_year"] = pd.to_datetime(df["Start_Time"]).dt.strftime('%u').astype(int)
    df["week"] = pd.to_datetime(df["Start_Time"]).dt.strftime('%W').astype(int)


In [1099]:
    # create zip code feature for border quarter and student quarters
    df["Borderdistrict"] = df["Zip_codes"].isin([60314, 60598, 60594, 63067, 60316, 60389])
    df["Student_quarter"] = df["Zip_codes"].isin([60385, 60487, 60385, 60326, 60486, 6028, 60327, 60488, 60487, 65929])

    # Set values to int
    df["Student_quarter"] = df["Student_quarter"].astype(int)
    df["Borderdistrict"] = df["Borderdistrict"].astype(int)
    df["Weekday"] = df["Weekday"].astype(int)

   

In [1100]:
    # create dummy featurre out of month 
    month = pd.get_dummies(df["month"], drop_first=True)
    month.columns = ["FE", "MA", "AP", "MA", "JU", "AU", "SE", "OC", "NO", "DE"]
    df = df.join(month)

In [1101]:
# create zipcode feature 

zipcodes = df["Zip_codes"].value_counts()

for plz in zipcodes.index:
    for i in range(0,52):
        value = df[(df["uni"] == 1) & (df["Zip_codes"] == plz) & (df["week"] < i)]["uni"].count()/(df[(df["Zip_codes"] == plz) & (df["week"] < i)])["uni"].count()
        df.loc[df[(df["Zip_codes"] == plz) & (df["week"] == i)].index,"zip_pro"] = value



df.fillna(value=0, inplace= True )

C:\Users\manue\miniconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys


In [1102]:
hour = df["hour"].value_counts()

for hour in hour.index:
    for i in range(0,52):
        value = df[(df["uni"] == 1) & (df["hour"] == hour) & (df["month"] < i)]["uni"].count()/(df[(df["hour"] == hour) & (df["month"] < i)])["uni"].count()
        df.loc[df[(df["hour"] == hour) & (df["month"] == i)].index,"hour_pro"] = value


df.fillna(value=0, inplace= True )


C:\Users\manue\miniconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


In [1103]:
# create station feature 
Start_Station = df["Start_Station"].value_counts()

for std in Start_Station.index:
    for i in range(0,52):
        value = df[(df["uni"] == 1) & (df["Start_Station"] == std) & (df["week"] < i) ]["uni"].count()/(df[(df["Start_Station"] == std) & (df["week"] < i)])["uni"].count()
        df.loc[df[(df["Start_Station"] == std) & (df["week"] == i)].index,"Start_Station_pro"] = value


hour = df["hour"].value_counts().index



C:\Users\manue\miniconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  


In [1115]:
# set target and preditor values
#Y = df["uni"]
X = df.drop(columns= ["uni"], inplace=True)

,0,end_d,Bike_number,Start_Time,End_time,Start_Longitude,Start_Latitude,End_Longitude,End_Latitude,Start_Station,...,MA,JU,AU,SE,OC,NO,DE,zip_pro,hour_pro,Start_Station_pro
13,15.685492,15.576618,11451,2019-05-30 15:42:00,2019-05-30 17:22:00,8.553437,50.101748,8.554450,50.101711,4291.0,...,1,0,0,0,0,0,0,0.833333,0.397129,0.823529
19571,15.685492,15.572969,38089,2019-05-30 15:43:00,2019-05-30 17:20:00,8.553437,50.101748,8.554479,50.101733,4291.0,...,1,0,0,0,0,0,0,0.833333,0.397129,0.823529
28319,4.388720,8.134838,38137,2019-05-30 15:42:00,2019-05-30 16:08:00,8.659075,50.108778,8.637696,50.081042,0.0,...,1,0,0,0,0,0,0,0.464106,0.397129,0.513687
39943,4.268353,6.818889,38198,2019-05-30 15:05:00,2019-05-30 15:16:00,8.668331,50.100153,8.649362,50.086095,4209.0,...,1,0,0,0,0,0,0,0.547325,0.397129,0.417910
52183,2.053770,6.069914,38267,2019-05-30 15:06:00,2019-05-30 15:28:00,8.679049,50.117486,8.643708,50.104957,4237.0,...,1,0,0,0,0,0,0,0.263397,0.397129,0.175182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152507,2.673466,2.551191,38993,2019-01-21 10:00:00,2019-01-21 10:09:00,8.695486,50.106433,8.707100,50.112704,4130.0,...,0,0,0,0,0,0,0,0.000000,0.000000,NaN
153342,1.787325,2.878442,38995,2019-12-25 12:13:00,2019-12-25 12:23:00,8.687546,50.115009,8.671153,50.115329,4247.0,...,0,0,0,0,0,0,1,0.281842,0.401812,0.293561
153360,3.078361,1.829000,38996,2019-02-14 02:06:00,2019-02-14 02:29:00,8.674521,50.109030,8.708392,50.126528,4242.0,...,0,0,0,0,0,0,0,0.483607,0.666667,0.666667
153682,0.820865,1.297919,38997,2019-04-08 00:27:00,2019-04-08 00:33:00,8.692680,50.123068,8.701279,50.122879,4250.0,...,0,0,0,0,0,0,0,0.096429,0.208333,0.040000


In [1135]:
X

,Weekday,weather,Borderdistrict,Student_quarter,FE,MA,AP,MA,JU,AU,SE,OC,NO,DE,zip_pro,Start_Station_pro
13,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0.833333,0.823529
19571,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0.833333,0.823529
28319,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0.464106,0.513687
39943,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0.547325,0.417910
52183,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0.263397,0.175182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152507,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0.000000,NaN
153342,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0.281842,0.293561
153360,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.483607,0.666667
153682,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0.096429,0.040000


In [1137]:



X = df

  
X["Start_Station_pro"].round(3) 
X["zip_pro"].round(3) 
# drop the other columns
X.drop(columns=["Start_Longitude", "Start_Latitude"], inplace=True)


KeyError: "['Start_Longitude' 'Start_Latitude'] not found in axis"

In [1122]:
x

0.418388037004182

In [1068]:
# initialte nn with 5 inputs and hidden layer 
model = keras.Sequential(
            [layers.Dense(17, activation="sigmoid", input_shape=[x_train.shape[1]]),
            layers.Dropout(0.1),
            layers.Dense(17, activation="sigmoid", ),
            layers.Dropout(0.1),
            layers.Dense(17, activation="sigmoid", ),
            layers.Dropout(0.1),
            layers.Dense(17, activation="sigmoid", ),
            layers.Dropout(0.1),
            layers.Dense(1)])

    optimizer = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss='rmse',
                optimizer=optimizer,
                metrics=["mae"])

[[-0.87248821  0.85109066  0.61819367 ... -1.4108619  -0.21781169
  -1.81286837]
 [ 1.14614729  0.00580377 -0.76068345 ...  0.23937408 -0.16506515
   0.71304786]
 [-0.87248821  0.63473881 -0.08103681 ... -0.81069628 -1.15572055
  -0.68504062]
 ...
 [ 1.14614729 -1.85952442 -1.24071169 ...  1.85767502 -0.35282717
   1.80775572]
 [-0.87248821  0.48553065  2.24002669 ...  0.79550228 -1.89313021
   0.52209104]
 [-0.87248821 -1.21725107  0.30097309 ...  1.38278062 -1.61210312
   0.38119862]]


In [1107]:
# set parameter for nn epochs and batch size
epochs = 20
batch_size = 5000
    
history = model.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>

In [ ]:
    # create confusion matrix
    print(confusion_matrix(Y, y_pred))

    history_df = pd.DataFrame(model.history)
    history_df

    root_metrics_df = history_df[["mse", "val_mse"]].apply(np.sqrt)
    root_metrics_df.rename({"mse": "rmse", "val_mse": "val_rmse"}, axis=1, inplace=True)
    root_metrics_df

    #create loss plot for epochs
    fig, ax = plt.subplots(1, 1, figsize=(12, 4), dpi=200)

    ax.plot(root_metrics_df["rmse"])

    ax.set_xlabel("Epochs")
    ax.set_ylabel("Root Mean Squared Error")

    ax.set_xlim([0, 20 - 1])

    plt.show()